In [ ]:
"""
notice
이상하게 매월 11-15일정도 데이터가 크롤링이 잘 안됩니다.
현재 데이터 다 긁은 다음에 11일부터 15일은 다른파일로 저장해서 합쳤습니다.
1회용 크롤러라 업데이트 없을 예정입니다.
어떻게 짰는지만 참고해주세요

"""

from selenium import webdriver
import time
import os, sys
from datetime import timedelta, date
import re

fname = "seoul_temp1.txt"
log_fname = "seoul_temp_log.txt"

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

def main():
    fname = "seoul_temp1.txt"
    if os.path.exists(fname):
        os.remove(fname)
    else:
        fname = input("파일이름을 입력해주세요(ex. out1.txt)")

    # driver 오픈
    driver = webdriver.Chrome("""C:\chromedriver_win32\chromedriver.exe""")
    driver.implicitly_wait(3)
    driver.get('https://data.kma.go.kr/data/grnd/selectAsosRltmList.do?pgmNo=36')

    el = driver.find_element_by_id('dataFormCd')
    for option in el.find_elements_by_tag_name('option'):
        if option.text == '분 자료':
            option.click()
            break
    driver.find_element_by_id('btnStn1').click()
    time.sleep(3)
    driver.find_element_by_id('ztree_2_switch').click()
    time.sleep(3)
    driver.find_element_by_id('ztree_4_check').click()
    driver.find_element_by_class_name('btn-sitetree-complete').click()

    driver.find_element_by_id('gubun').click()
    time.sleep(3)
    driver.find_element_by_id('ztree_3_check').click()

    driver.find_element_by_class_name('btn-sitetree-complete').click()

    el = driver.find_element_by_id('schListCnt')
    for option in el.find_elements_by_tag_name('option'):
        if option.text == '100':
            option.click()
            break

    start_date = date(2017, 1, 1)
    end_date = date(2017, 4, 1)
    for single_date in daterange(start_date, end_date):
        print(single_date.strftime("%Y-%m-%d"))
        yr_txt = single_date.strftime("%Y")
        mon_txt = str(int(single_date.strftime("%m")))+'월'
        day_txt = str(int(single_date.strftime("%d")))
        print(day_txt)

        #시작날짜설정
        driver.find_element_by_id('startDt').click()

        # 연도설정
        el = driver.find_element_by_class_name('ui-datepicker-year')
        for option in el.find_elements_by_tag_name('option'):
            if option.text == yr_txt:
                option.click()
                break
        # 월 설정
        el = driver.find_element_by_class_name('ui-datepicker-month')
        for option in el.find_elements_by_tag_name('option'):
            if option.text == mon_txt:
                option.click()
                break
        # 일 설정
        driver.find_element_by_link_text(day_txt).click()

        time.sleep(2)

        # 종료날짜설정
        driver.find_element_by_id('endDt').click()

        # 연도설정
        el = driver.find_element_by_class_name('ui-datepicker-year')
        for option in el.find_elements_by_tag_name('option'):
            if option.text == yr_txt:
                option.click()
                break
        # 월 설정
        el = driver.find_element_by_class_name('ui-datepicker-month')
        for option in el.find_elements_by_tag_name('option'):
            if option.text == mon_txt:
                option.click()
                break
        # 일 설정
        driver.find_element_by_link_text(day_txt).click()

        driver.find_element_by_class_name('addBtn').click()

        time.sleep(3)
        
        el = driver.find_elements_by_class_name('btn-area')[2]
        s = el.find_element_by_tag_name('strong').text
        n = int(re.findall('\d+', s)[0])
        if(n!=1440):
            with open(log_fname, 'w') as f:
                f.write(single_date.strftime("%Y-%m-%d")+' 데이터 개수이상(%d)' % n)
                print(single_date.strftime("%Y-%m-%d")+' 데이터의 개수이상(%d)' % n)
        
        w_data = []

        n_page = n//100 + 1
        for i in range(2,n_page+2):
            for tr in driver.find_elements_by_id('contentsList'):
                tds = tr.find_elements_by_tag_name('td')
                if tds:
                    w_data+= [td.text for td in tds]
            if(i==n_page+1): break

            bd = driver.find_element_by_class_name('board-nav')
            if(i == 11):
                bd.find_element_by_class_name('next_page').click()
            else:
                bd.find_element_by_link_text(str(i)).click()
        
        time.sleep(2)
        with open(fname,'a') as f:
            c = int(len(w_data)/3)
            for i in range(c):
                f.write('\t'.join(w_data[i*3:i*3+3]))
                f.write('\n')

if __name__ == "__main__":
    main()